In [1]:
# Parâmetros do pipeline (serão sobrescritos pelo SageMaker)
bucket_name = 'experimento-lucas-barbosa'
input_config_path = 'config.json'
input_metricas_path = 'metricas_completas.csv'
pasta_logs = 'logs/'
pasta_processed = 'processed/'

In [2]:
import pandas as pd
import boto3
import json
import os
from datetime import datetime

In [3]:
print(f"Início do salvamento: {datetime.now()}")
print(f"Parâmetros recebidos:")
print(f"  bucket_name: {bucket_name}")
print(f"  input_config_path: {input_config_path}")
print(f"  input_metricas_path: {input_metricas_path}")
print(f"  pasta_logs: {pasta_logs}")
print(f"  pasta_processed: {pasta_processed}")

Início do salvamento: 2025-07-21 18:40:29.733364
Parâmetros recebidos:
  bucket_name: experimento-lucas-barbosa
  input_config_path: config.json
  input_metricas_path: metricas_completas.csv
  pasta_logs: logs/
  pasta_processed: processed/


In [4]:
# Baixar arquivos do S3
s3 = boto3.client('s3')

s3.download_file(bucket_name, 'config/config.json', 'config.json')
s3.download_file(bucket_name, 'processed/metricas_completas.csv', 'metricas_completas.csv')

print('✓ Arquivos baixados com sucesso!')

✓ Arquivos baixados com sucesso!


In [5]:
# Carregar configurações
with open('config.json') as f:
    config = json.load(f)

# Definir paths
BUCKET = bucket_name
RAW_PREF = config['pasta_raw']
PROC_PREF = pasta_processed
ARCH_PREF = config['pasta_archive']
ERROR_PREF = config.get('pasta_error', 'raw/error/')
LOGS_PREF = pasta_logs

print('Configurações carregadas:')
print(f'  Bucket: {BUCKET}')
print(f'  Pasta raw: {RAW_PREF}')
print(f'  Pasta processed: {PROC_PREF}')
print(f'  Pasta archive: {ARCH_PREF}')
print(f'  Pasta error: {ERROR_PREF}')
print(f'  Pasta logs: {LOGS_PREF}')

Configurações carregadas:
  Bucket: experimento-lucas-barbosa
  Pasta raw: raw/
  Pasta processed: processed/
  Pasta archive: archive/
  Pasta error: raw/error/
  Pasta logs: logs/


In [6]:
# Carregar métricas completas
df_meta = pd.read_csv('metricas_completas.csv')
print(f'✓ Métricas carregadas: {len(df_meta)} linhas')
print(f'  Colunas: {list(df_meta.columns)}')

if len(df_meta) == 0:
    raise ValueError('Arquivo de métricas está vazio')

# Obter lista de arquivos processados com sucesso
arquivos_processados = df_meta['arquivo_origem'].unique().tolist()
print(f'  Arquivos processados com sucesso: {len(arquivos_processados)}')
print(f'  Lista: {arquivos_processados}')

✓ Métricas carregadas: 0 linhas
  Colunas: ['id_experimento', 'grupo', 'metrica', 'valor', 'timestamp', 'arquivo_origem']


ValueError: Arquivo de métricas está vazio

In [ ]:
# Preparar upload
id_experimento = df_meta['id_experimento'].iloc[0]
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
nome_meta = f'{id_experimento}_metricas_{timestamp}.csv'
key_meta = f'{PROC_PREF}{nome_meta}'
print(f'Arquivo de saída: {nome_meta}')
print(f'Chave S3: {key_meta}')

In [ ]:
# Salvar métricas finais na pasta processed/
df_meta.to_csv(f'/tmp/{nome_meta}', index=False)
s3.upload_file(f'/tmp/{nome_meta}', BUCKET, key_meta)
print(f'✓ Métricas enviadas para s3://{BUCKET}/{key_meta}')

In [ ]:
# Classificar e mover arquivos
print('\n🗂️ Classificando e movendo arquivos...')
print(f'Arquivos que foram processados com sucesso: {arquivos_processados}')

# Listar arquivos na pasta raw/
response = s3.list_objects_v2(Bucket=BUCKET, Prefix=RAW_PREF)
arquivos_movidos = []
arquivos_com_erro = []

# Verificar se existem arquivos
if 'Contents' in response and len(response['Contents']) > 0:
    print(f'Encontrados {len(response["Contents"])} objetos na pasta raw/')
    
    for obj in response['Contents']:
        arquivo_key = obj['Key']
        
        # Pular pastas e arquivos já em subpastas
        if arquivo_key.endswith('/') or arquivo_key == RAW_PREF or 'error/' in arquivo_key:
            print(f'  Pulando: {arquivo_key} (pasta ou já processado)')
            continue
            
        # Extrair nome do arquivo
        nome_arquivo = arquivo_key.replace(RAW_PREF, '')
        if not nome_arquivo:  # Se ficou vazio após replace
            continue
            
        print(f'Analisando arquivo: {nome_arquivo}')
        
        # Verificar se foi processado com sucesso
        if nome_arquivo in arquivos_processados:
            # SUCESSO: Mover para archive/
            destino_key = f"{ARCH_PREF}{nome_arquivo.replace('.csv', f'_{timestamp}.csv')}"
            
            s3.copy_object(
                Bucket=BUCKET,
                CopySource={'Bucket': BUCKET, 'Key': arquivo_key},
                Key=destino_key
            )
            s3.delete_object(Bucket=BUCKET, Key=arquivo_key)
            
            arquivos_movidos.append({
                'nome': nome_arquivo,
                'origem': arquivo_key,
                'destino': destino_key,
                'status': 'processado_com_sucesso'
            })
            
            print(f'  ✅ {nome_arquivo} → archive/')
            
        else:
            # ERRO: Mover para raw/error/
            destino_key = f"{ERROR_PREF}{nome_arquivo.replace('.csv', f'_erro_{timestamp}.csv')}"
            
            s3.copy_object(
                Bucket=BUCKET,
                CopySource={'Bucket': BUCKET, 'Key': arquivo_key},
                Key=destino_key
            )
            s3.delete_object(Bucket=BUCKET, Key=arquivo_key)
            
            arquivos_com_erro.append({
                'nome': nome_arquivo,
                'origem': arquivo_key,
                'destino': destino_key,
                'status': 'erro_processamento',
                'motivo': 'nao_processado_nas_metricas'
            })
            
            print(f'  ❌ {nome_arquivo} → raw/error/ (não foi processado)')
    
    print(f'\n📊 Resumo da movimentação:')
    print(f'  ✅ Arquivos processados: {len(arquivos_movidos)}')
    print(f'  ❌ Arquivos com erro: {len(arquivos_com_erro)}')
    
else:
    print('⚠️ Pasta raw/ está vazia - nenhum arquivo para processar')
    print('   Isso é normal se todos os arquivos já foram processados anteriormente')
    arquivos_movidos = []
    arquivos_com_erro = []

In [ ]:
# Salvar log detalhado
log = {
    'timestamp': datetime.now().isoformat(),
    'id_experimento': id_experimento,
    'metrics_key': key_meta,
    'metrics_file': nome_meta,
    'total_linhas_metricas': len(df_meta),
    'grupos': list(df_meta['grupo'].unique()),
    'metricas': list(df_meta['metrica'].unique()),
    'arquivos_processados_com_sucesso': arquivos_movidos,
    'arquivos_com_erro': arquivos_com_erro,
    'total_arquivos_processados': len(arquivos_movidos),
    'total_arquivos_com_erro': len(arquivos_com_erro),
    'bucket': BUCKET,
    'pastas': {
        'raw': RAW_PREF,
        'processed': PROC_PREF,
        'archive': ARCH_PREF,
        'error': ERROR_PREF,
        'logs': LOGS_PREF
    },
    'status': 'sucesso'
}
log_key = f'{LOGS_PREF}processamento_{id_experimento}_{timestamp}.json'
s3.put_object(Bucket=BUCKET, Key=log_key, Body=json.dumps(log, indent=2, ensure_ascii=False), ContentType='application/json')
print(f'✓ Log detalhado salvo em s3://{BUCKET}/{log_key}')
print(f'\n🎉 Processamento concluído com sucesso!')
print(f'   📊 Métricas salvas: {key_meta}')
print(f'   📁 Arquivos arquivados: {len(arquivos_movidos)}')
print(f'   ❌ Arquivos com erro: {len(arquivos_com_erro)}')
print(f'   📝 Log criado: {log_key}')